# Transformations

The aim of this notebook is to test different setups with transformations using cross-validation and measure top f1 score over all epochs and folds.

## Imports

In [1]:
import sys
sys.path.append("../")

In [2]:
import os
import cv2
import torch
import albumentations as A
import pandas as pd
import numpy as np
import segmentation_models_pytorch as smp

from sklearn.model_selection import KFold
from scripts.preprocessing import RoadDataset, split_data
from scripts.training import train_model
from torch.utils.data import DataLoader, SubsetRandomSampler

## Data

In [3]:
# specify root path and train directory
ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [4]:
# image paths so that all the images are used for train dataset (no test set for cv due to small training set)
image_path_train, _, mask_path_train, _ = split_data(train_directory, test_size=0)

# create train Dataset without transformations for now
train_dataset = RoadDataset(image_path_train, mask_path_train)

## Transformations

Define transformations we'll use in evaluating the performance of the model.

In [5]:
base_tf = [A.Resize(height=512, width=512, always_apply=True)]

tf_flip = [A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5)]
tf_rotate = A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse")
tf_brightness = A.RandomBrightnessContrast(p=0.5)
tf_snow = A.RandomSnow(p=0.1)

added_tfs = np.array([tf_flip, tf_rotate, tf_brightness, tf_snow], dtype=object)

In [31]:
# initiate the setups for transformations
col_names = ['base', 'flip', 'rotate', 'brightness', 'snow']
masks = [
    [False, False, False, False],
    [True, False, False, False],
    [True, True, False, False],
    [True, True, True, False],
    [True, True, True, True],
]

## Hyperparameters

Since our aim is to see, how different transformations influence the training, we fix the model, epochs and batch sizes.

In [10]:
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'

SEED = 13
BATCH_SIZE = 4
K_FOLD = 5
N_CPU = os.cpu_count()
N_EPOCHS = 100

LOADER_PARAMS = {
    'batch_size': BATCH_SIZE, 
    'num_workers': N_CPU, 
    'persistent_workers': True
}

## Cross-Validation

In [ ]:
metric_dict = {}

for setup in masks:
    
    # get the picked tfs as list
    tfs = added_tfs[np.array(setup)]
    added_tfs = pd.Series(tfs).explode().tolist()
    
    k_fold = KFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)
    
    # Record K-fold results in a (K_FOLD, num_epoch) matrix
    training_f1_matrix = []
    validation_f1_matrix = []
    
    train_tf = A.Compose(base_tf + added_tfs)
    valid_tf = A.Compose(base_tf)
    
    # Get training and validation indices 
    for fold, (train_idx, val_idx) in enumerate(k_fold.split(train_dataset)):
        
        # Create training and validation loaders by providing current K-Fold train/validation indices to Sampler
        train_loader = DataLoader(train_dataset.set_tf(train_tf), sampler=SubsetRandomSampler(train_idx), **LOADER_PARAMS)
        valid_loader = DataLoader(train_dataset.set_tf(valid_tf), sampler=SubsetRandomSampler(val_idx), **LOADER_PARAMS)
        
        # Initialize model
        model_ = smp.create_model("FPN", encoder_name=ENCODER, encoder_weights=ENCODER_WEIGHTS)
        criterion_ = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
        optimizer_ = torch.optim.Adam(model_.parameters(), lr=0.0005)
        scheduler_ = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer_,
            T_max=(len(train_loader.dataset) * N_EPOCHS) // train_loader.batch_size,
        )  
    
        # Train model
        train_losses, valid_losses, train_f1s, valid_f1s = train_model(
            model_, (train_loader, valid_loader), criterion_, optimizer_, scheduler_, N_EPOCHS
        )
        
        # Save epoch results
        training_f1_matrix.append(train_f1s)
        validation_f1_matrix.append(valid_f1s)
        
    metric_dict[str(setup)] = validation_f1_matrix

## Metrics

In [23]:
f1s = []
std_devs = []

for matrix in metric_dict.values():
    mean_per_epoch = matrix.mean(axis=0)
    
    best_epoch = mean_per_epoch.argmax()
    best_f1 = mean_per_epoch.max()
    
    f1s.append(mean_per_epoch.max())
    std_devs.append(matrix[:, best_epoch].std())

In [33]:
final_df = pd.DataFrame(index=metric_dict.keys(), columns=['top-f1', 'std'], data=np.array([f1s, std_devs]).T)

In [34]:
final_df

,top-f1,std
"[False, False, False, False]",0.801848,0.216771
"[True, False, False, False]",0.809387,0.054450
"[True, True, False, False]",0.828954,0.127005
"[True, True, True, False]",0.839682,0.218645
"[True, True, True, True]",0.938499,0.038345
